# Feb 26, 2023: copy Hadi's columnar brain parcellations

In [1]:
import os
import numpy as np 
import pandas as pd 
from tqdm import tqdm

from allensdk.core.mouse_connectivity_cache import (
    MouseConnectivityCache,
    MouseConnectivityApi
)

import glob 

import ants
import seaborn as sns 

In [2]:
nrois = 192 #512 # 192 # 128
# do the following in terminal:
# cd ~/mouse_dataset/allen_atlas_ccfv3/hadi/parcellation
# scp -r lceuser@kaba.umd.edu:/home/hadi/Documents/Ca-fMRI/processed/norm-global_parcel-columnar_n-192\*3/parcellation ./parcellation_n-192*3/
# scp -r lceuser@kaba.umd.edu:/home/hadi/Documents/Ca-fMRI/processed/norm-global_parcel-columnar_n-192\*3/roi_lookup.npy ./parcellation_n-192*3/

In [3]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

In [4]:
BASE_path = f'{os.environ["HOME"]}/mouse_dataset'
HADI_PARCELS_path = f'{BASE_path}/allen_atlas_ccfv3/hadi/parcellation/parcellation_n-{nrois}*3'
PARCELS_path = f'{BASE_path}/parcels'

In [5]:
h_parcels = np.load(f'{HADI_PARCELS_path}/brain_100um.npy')
h_parcels.shape

(132, 80, 114)

In [6]:
h_parcels_ctx = np.load(f'{HADI_PARCELS_path}/cortex_100um.npy')
np.unique(h_parcels_ctx)

array([   0,    1,    2, ..., 1150, 1151, 1152], dtype=uint32)

In [7]:
roi_lookup = np.load(f'{HADI_PARCELS_path}/roi_lookup.npy', allow_pickle=True).item()
len(roi_lookup['ca2'])

226

In [8]:
# whole brain
parcels_whl = h_parcels

# cortex
parcels_ctx = np.zeros_like(h_parcels, dtype=np.int64)

# calcium parcels
parcels_ca2 = np.zeros_like(h_parcels, dtype=np.int64)
for roi in roi_lookup['ca2'].values():
    parcels_ca2 += (h_parcels == roi) * roi

---

In [9]:
def to_nifti(args, img, print_=True):
    img = img.transpose(2, 0, 1)
    img = img[:,:,::-1]
    img = np.pad(
        img, 
        pad_width=((2, 2), (4, 24), (8, 2)), 
        mode='constant',
        constant_values=((0, 0), (0, 0), (0, 0))
        )
    if print_: print(img.dtype, img.shape)
    ndims = len(img.shape)
    ants_img = ants.from_numpy(
        data=img.astype(np.float32), 
        origin=[6.4, -13.2, -7.8],
        spacing=[0.1]*ndims,
    )
    return ants_img

In [10]:
args.atlas_path = f'{os.environ["HOME"]}/mouse_dataset/allen_atlas_ccfv3'
args.mcc_path = f'{args.atlas_path}/MouseConnectivity'
mcc = MouseConnectivityCache(
    resolution=100, # in micro meters (um)
    ccf_version=MouseConnectivityApi().CCF_2017,
    manifest_file=f'{args.mcc_path}/manifest.json',
)
AVGT, metaAVGT = mcc.get_template_volume()
ANO, metaANO = mcc.get_annotation_volume()
AVGT = AVGT.astype(np.float32)
ANO = ANO.astype(np.uint32)
print(AVGT.shape, ANO.shape)

STree = mcc.get_structure_tree()
STree_df = pd.DataFrame(STree.nodes()) 
# for idx in STree_df.id.to_list():
#     try: 
#         mcc.get_structure_mask(structure_id=idx) 
#     except:
#         pass

(132, 80, 114) (132, 80, 114)


In [11]:
# templates in nifti
n162_100um_template = f'{os.environ["HOME"]}/mouse_dataset/gabe_symmetric_N162/Symmetric_N162_0.10_RAS.nii.gz'
n162_100um_template = ants.image_read(n162_100um_template)
print(n162_100um_template.numpy().dtype, n162_100um_template.numpy().shape)

allen_template = to_nifti(args, AVGT)

float32 (118, 160, 90)
float32 (118, 160, 90)


In [12]:
# for reproducible registration
os.system('export ITK_GLOBAL_DEFAULT_NUMBER_OF_THREADS=1')
os.system('export ANTS_RANDOM_SEED=1')

tx = ants.registration(
    fixed=n162_100um_template,
    moving=allen_template,
    type_of_transform=('SyN'),
    random_seed=args.SEED,
)

def transform(args, img):
    img_tx = ants.apply_transforms(
        fixed=n162_100um_template,
        moving=img,
        transformlist=tx['fwdtransforms'],
        interpolator='genericLabel',
    )
    return img_tx
    
allen_template_tx = transform(args, img=allen_template)

In [13]:
# resampling to 0.2mm resolution
n162_200um_template = f'{os.environ["HOME"]}/mouse_dataset/gabe_symmetric_N162/Symmetric_N162_0.20_RAS.nii.gz'
n162_200um_template = ants.image_read(n162_200um_template)

def resample(args, target, img):
    img_rs = ants.resample_image_to_target(
        image=img,
        target=target,
        interp_type='genericLabel',
    )
    img_rs = img_rs.new_image_like(
        data=img_rs.numpy() * (target.numpy() > 0)
    )
    print(img_rs.numpy().shape)
    return img_rs
    
allen_template_tx_rs = resample(args, target=n162_200um_template, img=allen_template_tx)

(60, 81, 46)


In [14]:
# common brain mask (across subs)
BASE_path = f'{os.environ["HOME"]}/mouse_dataset'
all_files_path = f'{BASE_path}/voxel/all_file_collections'
all_files = os.listdir(all_files_path)

# cmask : common brain mask
for idx, files in tqdm(enumerate(all_files[:])):
    if idx == 0:
        with open(f'{all_files_path}/{files}', 'r') as f:
            cmask_img = ants.image_read(f.readlines()[1][:-1])
        cmask = cmask_img.numpy()
    else:
        with open(f'{all_files_path}/{files}', 'r') as f:
            cmask *= ants.image_read(f.readlines()[1][:-1]).numpy()
cmask_img = cmask_img.new_image_like(cmask)
cmask_img.to_filename(
    f'{BASE_path}/voxel/common_brain_mask.nii.gz'
)

116it [00:00, 257.68it/s]


---

In [15]:
def transform_parcels(args, parcels,):
    # all transformations
    parcels_img = to_nifti(args, parcels)
    parcels_img_tx = transform(args, img=parcels_img)
    parcels_img_tx_rs = resample(args, target=n162_200um_template, img=parcels_img_tx)
    parcels_img_tx_rs_cm = resample(args, target=cmask_img, img=parcels_img_tx_rs)
    
    # description of the parcellation
    args.num_rois = len(np.unique(parcels_img_tx_rs_cm.numpy())[1:])
    DESC = (
        f'type-{args.type}'
        f'_size-{args.roi_size}'
        f'_symm-{args.maintain_symmetry}'
        f'_braindiv-{args.brain_div}'
        f'_nrois-{args.num_rois}'
    )
    parcels_file = f'{PARCELS_path}/{DESC}_desc-parcels.nii.gz'
    labels_file = f'{PARCELS_path}/{DESC}_desc-labels.txt'
    
    # save
    parcels_img_tx_rs_cm.to_filename(parcels_file)
    
    # roi labels
    cmd = (
        f'3dROIstats -overwrite '
        f'-quiet '
        f'-mask {parcels_file} '
        f'{parcels_file} > {labels_file}'
    )
    os.system(cmd)

    # done
    return parcels_img_tx_rs_cm

In [16]:
# args.type = 'columnar'
# args.roi_size = 'x'
# args.maintain_symmetry = True
# args.brain_div = 'whl'
# parcels_whl_img_tx_rs_cm = transform_parcels(args, parcels_whl,)

In [17]:
args.type = 'columnar'
args.roi_size = 'x'
args.maintain_symmetry = True
args.brain_div = 'ca2'
parcels_ca2_img_tx_rs_cm = transform_parcels(args, parcels_ca2,)

int64 (118, 160, 90)
(60, 81, 46)
(58, 79, 45)


---

In [18]:
# cd ~/mouse_dataset/allen_atlas_ccfv3/hadi/parcellation/parcellation_n-192*3
# mkdir -p bold
# cd bold
# scp -r lceuser@kaba.umd.edu:/home/hadi/Documents/Ca-fMRI/processed/norm-global_parcel-columnar_n-192*3/bold/*task-rest*rabies-hp* .

In [19]:
assert np.all(np.unique(parcels_ca2)[1:] == np.array(list(roi_lookup['ca2'].values())))

In [20]:
ca2 = roi_lookup['ca2']
k, v = list(ca2.keys()), list(ca2.values())
ca2_df = pd.DataFrame({
    'idx':k, 
    'roi':v
}).reset_index(drop=True)
# ca2_df

bold = roi_lookup['bold']
k, v = list(bold.keys()), list(bold.values())
bold_df = pd.DataFrame({
    'idx':k, 
    'roi':v,
}).reset_index(drop=True)
# bold_df

roi_idxs = bold_df[bold_df['roi'].isin(ca2_df['roi'].to_list())]['idx'].to_list()

In [21]:
TS_path = f'{BASE_path}/roi'
DESC = (
    f'type-{args.type}'
    f'_size-{args.roi_size}'
    f'_symm-{args.maintain_symmetry}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
)
TS_path = f'{TS_path}/{DESC}/roi_timeseries'
os.system(f'mkdir -p {TS_path}')

0

In [22]:
bold_files = glob.glob(f'{HADI_PARCELS_path}/bold/*', recursive=True)
for bold_file in bold_files:
    ts_file = '_'.join(bold_file.split('/')[-1].split('_')[:4] + [f'desc-roi-ts.txt'])
    bold_ts = np.load(bold_file)[roi_idxs, :].T
    np.savetxt(
        f'{TS_path}/{ts_file}',
        bold_ts,
        fmt='%.3e',
    )